# 5. Pokušaji Rešenja Problema

## Dva ključna pristupa poboljšanju performansi

---

Nakon uočavanja kritičnih problema u notebook-u 04, implementirana su dva glavna rešenja koja adresiraju identificirane probleme. Pristup je bio sistematičan - od kombinovanja sličnih modela do potpuno novog hijerarhijskog pristupa.

In [ ]:
# Import biblioteka
import sys
from pathlib import Path

# Dodaj src u path
current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'notebooks' else current_dir.parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

from utils.car_brands import MODEL_CONSOLIDATIONS
from hierarchical_model import HierarchicalCarClassifier

### Rešenje 1: Spajanje Sličnih Modela

Prvi korak u rešavanju problema prekomerne složenosti bila je **redukcija broja modela** kroz sistematsko kombinovanje sličnih varijanti.

#### 🔍 Identifikovani problemi:
Problem od 1358 različitih modela je bio previše za efikasno učenje zbog postojanja:

- **Varijacije po broju vrata**: "147_3_DOORS" i "147_5_DOORS" predstavljaju isti model automobila
- **Generacijske oznake**: "1_SERIES_E87" i "1_SERIES_F20" su različite generacije BMW 1 serije
- **Specijalne edicije**: "A4_DTM_EDITION" je samo specijalna verzija osnovnog "A4" modela
- **Duplikati sa različitim imenima**: "500C_ABARTH" i "500_ABARTH" označavaju isti automobil

#### ✅ Implementirano rešenje:
Kreiran je `MODEL_CONSOLIDATIONS` rečnik koji mapira preko 100 varijanti u osnovne modele, čime se:
- Smanjuje broj klasa za treniranje
- Poboljšava kvalitet oznaka
- Redukuje konfuziju između sličnih modela

In [ ]:
# Primeri spajanja modela iz src/utils/car_brands.py

spajanje_primeri = {
    # Varijacije po broju vrata
    '147_3_DOORS': '147',
    '147_5_DOORS': '147',
    
    # BMW generacije - sve se svode na osnovni model
    '1_SERIES_E87': '1_SERIES', 
    '1_SERIES_F20': '1_SERIES',
    '3_SERIES_E36': '3_SERIES',
    '3_SERIES_E46': '3_SERIES',
    '3_SERIES_E90': '3_SERIES',
    
    # Specijalne verzije se svode na osnovni model
    'A4_DTM_EDITION': 'A4',
    'A4_Avant': 'A4',
    'A4_ALLROAD': 'A4',
    
    # Duplikati sa različitim imenima
    '500C_ABARTH': '500_ABARTH',
    'COOPER_S': 'COOPER',
    'COUNTRYMAN_S': 'COUNTRYMAN'
}

print(f"📊 Primer spajanja: {len(spajanje_primeri)} mapiranja")
print(f"🔗 Ukupno spajanja u sistemu: {len(MODEL_CONSOLIDATIONS)}")
print(f"\n💡 Efekat: Smanjuje duplikovanje i poboljšava kvalitet podataka")

#### 📈 Rezultat spajanja modela:

Kroz proces spajanja modela postignuto je:

- **Smanjenje duplikata**: Preko 100 varijanti svedeno na osnovne modele
- **Poboljšanje balansa klasa**: Kombinovanjem varijanti povećava se broj uzoraka po osnovnom modelu
- **Konzistentnost oznaka**: Eliminisani su konflikte između različitih načina pisanja istog modela
- **Lakše treniranje**: Model više ne mora da razlikuje semantički identične varijante

Međutim, čak i nakon spajanja, problem od preko 1300 modela je i dalje bio previše složen za efikasno učenje.

In [ ]:
# Analiza uticaja spajanja modela

uticaj_spajanja = {
    'pre_spajanja': {
        'broj_modela': '~1400 (sa duplikatima)',
        'problem': 'mnoge varijante istog modela',
        'primer': '147_3_DOORS, 147_5_DOORS kao različite klase'
    },
    'posle_spajanja': {
        'broj_modela': '~1300 (konsolidovano)',
        'poboljsanje': 'svedeno na osnovne modele',
        'primer': 'sve varijante 147 → jedna klasa 147'
    },
    'kljucni_efekat': {
        'kvalitet': 'konzistentnije oznake',
        'balans': 'više uzoraka po osnovnom modelu',
        'jednostavnost': 'manje semantičkih duplikata'
    }
}

# Primer BMW serija pre i posle spajanja
bmw_primer = {
    'pre': ['1_SERIES', '1_SERIES_E87', '1_SERIES_F20', '1_SERIES_F40'],
    'posle': ['1_SERIES'],  # Sve varijante spojene u jednu
    'efekat': '4x više uzoraka za 1_SERIES klasu'
}

print("🔄 BMW 1 serija primer:")
print(f"Pre: {bmw_primer['pre']}")
print(f"Posle: {bmw_primer['posle']}")
print(f"Efekat: {bmw_primer['efekat']}")

### Rešenje 2: Hijerarhijski Pristup - Revolucionarna Promena

Kada standardni pristup sa više izlaza nije uspeo čak ni posle spajanja modela, implementiran je potpuno **novi hijerarhijski pristup** koji fundamentalno menja način rešavanja problema.

#### 🎯 Osnovna ideja hijerarhije:

Umesto pokušaja da model simultano nauči sve tri zadatka (proizvođač + model + godina), problem se **deli na logičke celine**:

1. **Faza 1 - Prepoznaj proizvođača**: BMW vs AUDI vs MERCEDES vs FORD...
2. **Faza 2 - Prepoznaj model za poznati proizvođač**: BMW → (1_SERIES, 3_SERIES, X5...)
3. **Faza 3 - Proceni godište**: Nezavisno od proizvođača i modela

#### 🧠 Ključne prednosti hijerarhijskog pristupa:

- **Dramatična redukcija složenosti**: Umesto 1300+ modela simultano, svaki proizvođač ima samo ~10-50 modela
- **Specijalizovani klasifikatori**: BMW klasifikator se fokusira **isključivo** na BMW modele
- **Prirodnost pristupa**: Odgovara tome kako ljudi prepoznaju automobile u realnosti
- **Postupno učenje**: Svaka faza se može trenirati i optimizovati nezavisno

In [ ]:
# Hijerarhijska arhitektura - osnovna struktura iz src/hierarchical_model.py

# Osnovna struktura HierarchicalCarClassifier
hijerarhijska_struktura = """
class HierarchicalCarClassifier(nn.Module):
    def __init__(self, num_makes, num_years, make_to_models, pretrained=True):
        super().__init__()
        
        # Faza 1: Klasifikator proizvođača (81 klasa)
        self.make_classifier = MakeClassifier(num_makes, pretrained)
        
        # Faza 2: Klasifikator modela (specifičan po proizvođaču)
        self.model_classifier = ModelClassifier(make_to_models, num_makes, pretrained)
        
        # Faza 3: Klasifikator godine (12 klasa)
        self.year_classifier = YearClassifier(num_years, pretrained)
    
    def forward(self, x, make_ids=None, make_names=None, stage='all'):
        results = {}
        
        # Izvršava se samo potrebna faza ili sve faze
        if stage in ['make', 'all']:
            results['make'] = self.make_classifier(x)
            
        if stage in ['model', 'all'] and make_ids is not None:
            results['model'] = self.model_classifier(x, make_ids, make_names)
                
        if stage in ['year', 'all']:
            results['year'] = self.year_classifier(x)
            
        return results
"""

print("🏗️ Hijerarhijska arhitektura:")
print(hijerarhijska_struktura)

### Postupno Treniranje - Četiri Kontrolisane Faze

Hijerarhijski model omogućava **stage-wise training** - kontrolisano učenje kroz četiri specijalizovane faze umesto jedne složene.

#### 📖 Faza 1 - Učenje Proizvođača (5 epoha):
- **Šta se trenira**: Samo `make_classifier` komponenta
- **Broj klasa**: 81 proizvođač (BMW, AUDI, MERCEDES...)
- **Zašto je lako**: Brendovi su vizuelno vrlo različiti
- **Brzina učenja**: 0.001 (standardna)

#### 🎯 Faza 2 - Učenje Modela (8 epoha):
- **Šta se trenira**: `model_classifier` sa **fiksiranim proizvođačem**
- **Ključno**: BMW klasifikator uči samo BMW modele (~20 klasa)
- **Ključno**: AUDI klasifikator uči samo AUDI modele (~15 klasa)
- **Efekat**: Potpuno eliminisana konfuzija između brendova
- **Brzina učenja**: 0.0005 (sporija, preciznija)

#### ⏰ Faza 3 - Učenje Godina (5 epoha):
- **Šta se trenira**: `year_classifier` potpuno nezavisno
- **Broj klasa**: 12 decenija
- **Prednost**: Decenije su vizuelno konzistentne kroz brendove
- **Brzina učenja**: 0.001 (standardna)

#### 🔄 Faza 4 - Finalno Podešavanje (10 epoha):
- **Šta se trenira**: **Ceo model** zajedno
- **Brzina učenja**: 0.0001 (vrlo spora - čuva naučeno)
- **Cilj**: Optimizuje koordinaciju između svih komponenti

In [ ]:
# Strategija postupnog treniranja - iz scripts/train_hierarchical.py

strategija_treniranja = {
    'faza_1_proizvodjac': {
        'epohe': 5,
        'brzina_ucenja': 0.001,
        'cilj': 'razlikovanje brendova',
        'klase': 81,
        'optimizuje': 'make_classifier.parameters()',
        'primer': 'BMW vs AUDI vs MERCEDES'
    },
    'faza_2_model': {
        'epohe': 8, 
        'brzina_ucenja': 0.0005,
        'cilj': 'modeli unutar poznatog brenda',
        'klase': '10-50 po proizvođaču',
        'optimizuje': 'model_classifier.parameters()',
        'primer': 'BMW: 1_SERIES vs 3_SERIES vs X5'
    },
    'faza_3_godina': {
        'epohe': 5,
        'brzina_ucenja': 0.001, 
        'cilj': 'procena starosti automobila',
        'klase': 12,
        'optimizuje': 'year_classifier.parameters()',
        'primer': '1990s vs 2000s vs 2010s'
    },
    'faza_4_sve': {
        'epohe': 10,
        'brzina_ucenja': 0.0001,
        'cilj': 'koordinacija svih komponenti',
        'klase': 'sve kombinovano',
        'optimizuje': 'ceo model zajedno',
        'primer': 'BMW 3_SERIES iz 2010s'
    }
}

# Primer redukcije složenosti
redukcija_slozonosti = {
    'standardni_pristup': {
        'klase_simultano': 1300,
        'problem': 'BMW 3_SERIES vs AUDI A4 vs FORD FOCUS simultano'
    },
    'hijerarhijski_pristup': {
        'faza_1': '81 brendova',
        'faza_2_bmw': '20 BMW modela',
        'faza_2_audi': '15 AUDI modela',
        'prednost': 'BMW klasifikator se NIKAD ne meša sa AUDI modelima'
    }
}

ukupno_epoha = sum(stage['epohe'] for stage in strategija_treniranja.values())
print(f"📚 Ukupno epoha: {ukupno_epoha} kroz 4 kontrolisane faze")
print(f"🎯 Redukcija složenosti: 1300+ → maks 50 klasa po fazi")
print(f"🧠 Ključno: Svatki brand ima svoj specijalizovani klasifikator")

### Napredna Arhitektura - ModelClassifier

Najsofisticiranija komponenta hijerarhijskog sistema je **ModelClassifier** koji implementira make-specific klasifikatore.

#### 🏗️ Tehnički izazovi ModelClassifier-a:

1. **Različit broj modela po proizvođaču**: BMW ima 20 modela, FIAT ima 8 modela
2. **Dinamički klasifikatori**: Potreban je odvojeni klasifikator za svaki brend
3. **Konzistentne dimenzije**: Batch processing zahteva uniforme tensor dimenzije
4. **Kontekst proizvođača**: Informacija o brendu mora biti ugrađena u model

#### ⚙️ Implementirano rešenje:

- **`nn.ModuleDict`**: Odvojeni klasifikator za svaki proizvođač
- **Make embedding**: 128D vektor reprezentacija proizvođača
- **Feature fusion**: Kombinovanje image features (2048D) + make context (128D)
- **Dynamic routing**: Automatsko biranje odgovarajućeg klasifikatora po uzorku
- **Padding strategy**: Izlazni tenzor sa maksimalnom veličinom, dopunjen nulama

In [ ]:
# ModelClassifier - ključni delovi implementacije

model_classifier_kod = """
class ModelClassifier(nn.Module):
    def __init__(self, make_to_models, num_makes, pretrained=True):
        super().__init__()
        
        # ResNet-50 za ekstraktovanje vizuelnih karakteristika
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-1])
        
        # Embedding za kontekst proizvođača
        self.make_embedding = nn.Embedding(num_makes, 128)
        
        # Odvojeni klasifikator za svakog proizvođača
        self.model_classifiers = nn.ModuleDict()
        for make, models in make_to_models.items():
            # Ulaz: slika (2048D) + proizvođač (128D) = 2176D
            self.model_classifiers[make] = nn.Linear(2048 + 128, len(models))
    
    def forward(self, x, make_ids, make_names):
        # Izvuci karakteristike slike
        features = self.feature_extractor(x)  # [batch, 2048]
        features = torch.flatten(features, 1)
        
        # Dobij kontekst proizvođača
        make_emb = self.make_embedding(make_ids)  # [batch, 128]
        
        # Kombinuj vizuelne i kontekstualne informacije
        combined = torch.cat([features, make_emb], dim=1)  # [batch, 2176]
        
        # Dinamički izbor klasifikatora za svaki uzorak
        max_models = max(len(models) for models in self.make_to_models.values())
        output = torch.zeros(batch_size, max_models, device=x.device)
        
        for i, make_name in enumerate(make_names):
            if make_name in self.model_classifiers:
                # Koristi odgovarajući klasifikator za ovaj brend
                classifier = self.model_classifiers[make_name]
                single_output = classifier(combined[i:i+1])
                output[i, :single_output.size(1)] = single_output[0]
        
        return output
"""

# Primer mapiranja proizvođač → modeli
make_to_models_primer = {
    'BMW': ['1_SERIES', '3_SERIES', '5_SERIES', 'X1', 'X3', 'X5', 'Z4'],        # 7 modela
    'AUDI': ['A3', 'A4', 'A6', 'A8', 'Q3', 'Q5', 'Q7', 'TT'],                  # 8 modela
    'MERCEDES': ['A_CLASS', 'C_CLASS', 'E_CLASS', 'S_CLASS', 'GLA', 'GLC'],     # 6 modela
    'FORD': ['FIESTA', 'FOCUS', 'MONDEO', 'KUGA'],                              # 4 modela
    'VOLKSWAGEN': ['GOLF', 'PASSAT', 'POLO', 'TIGUAN', 'TOURAN']                # 5 modela
}

print("🔧 ModelClassifier arhitektura:")
print(model_classifier_kod[:500] + "...")
print(f"\n📊 Primer specijalizacije po brendovima:")
for make, models in make_to_models_primer.items():
    print(f"{make}: {len(models)} modela - {models[:3]}...")

### Zaključak: Evolucija ka Hijerarhijskom Pristupu

Projekat je prošao kroz **sistematsku evoluciju rešenja**:

#### 🔄 Put do finalnog rešenja:
1. **Početni problem**: Standardni MultiOutputCarClassifier sa katastrofalnim rezultatima
2. **Prvo poboljšanje**: Spajanje sličnih modela za redukciju duplikata
3. **Revolucionarno rešenje**: Hijerarhijski pristup sa potpuno novom arhitekturom

#### 💡 Ključne lekcije:
- **Kompleksni problemi zahtevaju kreativne pristupe**: Standardni pristupi nisu uvek dovoljni
- **Prirodna hijerarhija je efikasnija**: Ljudi prvo vide brend, zatim model - model treba da radi isto
- **Postupno učenje omogućava kontrolu**: Stage-wise training daje bolju kontrolu nad procesom
- **Specijalizacija poboljšava performanse**: Make-specific klasifikatori su efikasniji od globalnih

#### 🚀 Očekivani rezultati hijerarhijskog pristupa:
**HierarchicalCarClassifier** sa strategijom postupnog treniranja predstavlja najobećavajući pristup jer:

- **Deli složen problem na prirodne celine** umesto forsiranja simultane optimizacije
- **Omogućava duboku specijalizaciju** kroz make-specific klasifikatore
- **Eliminiše cross-brand konfuziju** koja je mučila baseline model
- **Prirodniji je za domen automobila** i odgovara ljudskom načinu prepoznavanja

**Sledeći korak**: Evaluacija performansi hijerarhijskog pristupa i analiza poboljšanja u odnosu na baseline model.